Set theory is interesting.

It's a really cool thing that it builds up a ton of structure from such basic seeming components.

The concept of a set within a set is a bit unfamiliar from a programming perspective. This rarely comes up?
Typical set data structures require an ability to totally order or hash its elements. The subset relationship is almost a canonical example of a partial order. 
Hashing of sets is very interesting.



By using hash consing, we can give an arbitrary total ordering to each set as we build it, the order given by it's `id` identifiers. This ordering will not be stable between different runs.
In this way we can use the method of taking a sorted deduplicated tuple as being a canonical representative of it's set. Hashing this tuple in the usual way works fine.
Another approach is to consider using a hash combination function that respects the properties of the set datastructure. In other words the hash combination function is something like a homomorphism from sets to integers. An example might be `xor`, which is associative and commutative, just like set union.





In [15]:
from dataclasses import dataclass
from typing import Iterable
univ = {}
@dataclass(frozen=True)
class HashSet(): # do not use this constructor
    elems: tuple["HashSet", ...] # names. elem, mem, items   
    def __lt__(self, other) -> bool:
        return self.elems < other.elems
    def __hash__(self) -> int:
        return hash(id(self))
    def __eq__(self, other) -> bool: # fast equality via pointer equality
        return self is other
    def __iter__(self): # essentially enables a comprehension/separation operation
        return iter(self.elems)
    def __len__(self):
        return len(self.elems)
    def __repr__(self): # pretty printing
        return "{" + ",".join(map(repr, self.elems)) + "}"

# hmm. Could I use functools cache here? But maybe then it's hard to get the univ later
def hashset(x : Iterable[HashSet]) -> HashSet:
    """Smart constructor returns literally the same object if the same input is given."""
    x = tuple(sorted(set(x)))
    if x in univ:
        return univ[x]
    else:
        y = HashSet(x)
        univ[x] = y
        return y
    
emp = hashset([])
print(f"{emp=}")

# When we make the same hashset twice, they should be the same
x = hashset([])
y = hashset([])
print(f"{hashset([x]) is hashset([y, y])=}")


emp={}
hashset([x]) is hashset([y, y])=True


Union is a new primitive operation. Anything that needs to touch the `elems` field is peeking under the curtains. Whereas intersection is a derived operation because it can use comprehension. Is this true? Ehhhh. Kind of we have an ambient theory of lists and tuples. We can convert to them using a comprehension. Are lists, tuples, generators kind of like "classes"? They are HashSet like.

In [6]:

def union(x,y):
    return hashset(x.elems + y.elems)

def intersect(x:HashSet,y:HashSet) -> HashSet:
    return hashset([z for z in x if z in y])

def eats(x,y): # aka add
    return hashset((y,) + x.elems)


In [7]:
#one = hashset([emp])

def succ(x: HashSet) -> HashSet:
    return eats(x, x)
one = succ(emp)
one
two = succ(one)
two
three = succ(two)
print(f"{three=}")
print(f"{len(three)=}")

three={{},{{}},{{},{{}}}}
len(three)=3


In [ ]:

from functools import cache
#memoization

@cache
def from_int(n):
    assert n >= 0
    if n == 0:
        return emp
    return succ(from_int(n-1))

print(f"{from_int(3) == three=}")
from_int(4)

In [7]:
def reify() -> Hashset:
    return hashset(univ.values())

#reify()

In [8]:
def pred(z : Hashset) -> Hashset:
    return max(z.x, key=len)

pred(three) == two

True

In [16]:
import itertools
def power(s : HashSet) -> HashSet:
    # https://docs.python.org/3/library/itertools.html#itertools-recipes
    return hashset(itertools.chain.from_iterable(map(hashset, itertools.combinations(s.x, r)) for r in range(len(s)+1)))

power(power(emp))

wrap2 = hashset([hashset([emp])])

def closure(s : HashSet) -> HashSet:
    return reduce(union, [closure(x) for x in s])
    #pass #return hashset([ power(closure(x)) for x in s ])

def natlabel(s:HashSet) -> int:
    return sum(2**natlabel(x) for x in s)

natlabel(emp)
natlabel(one)
natlabel(wrap2)
natlabel(two)
natlabel(three)


NameError: name 'Hashset' is not defined

In [10]:
def plus(x: HashSet, y : HashSet):
    if x == emp:
        return y
    return hashset([plus(x,y) for x in x.x])

@cache
def plus(x:HashSet, y : HashSet):
    if x == emp:
        return y
    return hashset([plus(x,y) for x in x.x])


def pair(x,y):
    return hashset([hashset([x]), hashset([x,y])])

def fst(z):
    x,y = z.x
    if len(x) == 1:
        return x
    else:
        return y

def snd(z):
    x,y = z.x
    if len(y) == 1:
        x,y = y,x
    return y - x

NameError: name 'HashSet' is not defined

One of the uses of pairs is to encode functions and relations. A natural form of a finite function in python is a dictionary. Here we show the isomorphism

In [ ]:
def from_dict(d):
    return hashset([pair(k, v) for k,v in d.items()])
def to_dict(z):
    return {fst(x):snd(x) for x in z}
def domain(z):
    return hashset([fst(x) for x in z])
def codomain(z):
    return hashset([snd(x) for x in z])

Families



In [18]:
def choice(Xi):
    return hashset([pair(fst(x), snd(x).elems[0]) for x in Xi])

Can we talk about induction?

The system is missing the ability the talk about the hypothetical. A little bit of crazy talk, but maybe one way of doing this is using promises/futures. If a value is never forced, then it's contents do not matter. This is similar to inferring forall polymorphism or when a prolog query returns an unbound metavariable.

Can we include infinite sets? In some sense perhaps. I believe we can basically attach some ordinals. There are certain questions that won't be computable. `[from_int(i) for i integers()] in omega` is true, but python won't ever return true.

https://en.wikipedia.org/wiki/Von_Neumann_universe

set logic programming


There is of course an empty set.

Comprehension is allowed. We obviously need to perform comprehension over a known set, so it is separation

axiom of choice. We don't really need the axiom of choice since we don't have infinite things

Hereditarily Finite Sets
Paulson https://lawrencecpaulson.github.io/2022/02/23/Hereditarily_Finite.html


Non well founded sets like Graham's thing
Aczel
We can make loopy set structures if we take iterators / lazy data structures as our sets.

The laziness allows the set to be deep (infinite depth to set) or wide (infinite card set)






In [ ]:
%%file /tmp/set.tptp
cnf(emp_exist, axiom, ~elem(X,emp)).
fof(extension, axiom, (![X] : (elem(X,A) <=> elem(X,B)))) <=> A = B).
fof(power, )


